[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/EMQA/blob/main/EMQA_electricity_analysis/EMQA_electricity_analysis.ipynb)

# EMQA_electricity_analysis

**Romanian Electricity Price Statistics**

Exploratory analysis of Romanian day-ahead electricity prices: time series, distribution, returns, and monthly averages.

**Output:** `electricity_analysis.pdf`

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

plt.rcParams.update({
    'figure.facecolor': 'none',
    'axes.facecolor': 'none',
    'savefig.facecolor': 'none',
    'savefig.transparent': True,
    'axes.grid': False,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'font.size': 11,
    'figure.figsize': (12, 6),
})

COLORS = {
    'blue': '#1A3A6E', 'red': '#CD0000', 'green': '#2E7D32',
    'orange': '#E67E22', 'purple': '#8E44AD', 'gray': '#808080',
    'cyan': '#00BCD4', 'amber': '#B5853F'
}

def save_fig(fig, name):
    fig.savefig(name, bbox_inches='tight', transparent=True, dpi=300)
    print(f"Saved: {name}")


In [ ]:
url = 'https://raw.githubusercontent.com/QuantLet/EMQA/main/EMQA_electricity_analysis/ro_de_prices_extended.csv'
df = pd.read_csv(url, parse_dates=['date'], index_col='date')
prices = df['ro_price'].dropna()
print(f'Loaded {len(prices)} daily observations')
print(f'Date range: {prices.index[0].date()} to {prices.index[-1].date()}')
print(f'Mean: {prices.mean():.1f} EUR/MWh, Std: {prices.std():.1f}')


In [ ]:
print("Romanian Electricity Price — Descriptive Statistics")
print("=" * 50)
print(prices.describe())
print(f"\nSkewness:  {prices.skew():.4f}")
print(f"Kurtosis:  {prices.kurtosis():.4f}")
print(f"\nLog Returns:")
print(f"  Mean:     {returns.mean():.4f}%")
print(f"  Std:      {returns.std():.4f}%")
print(f"  Skewness: {returns.skew():.4f}")
print(f"  Kurtosis: {returns.kurtosis():.4f}")

In [ ]:
from scipy.stats import norm
import matplotlib.dates as mdates

fig, axes = plt.subplots(2, 2, figsize=(14, 9))

# Panel 1: Price time series
axes[0, 0].plot(prices.index, prices.values, color=COLORS['blue'], linewidth=0.6)
axes[0, 0].set_title('Romanian Day-Ahead Electricity Price', fontweight='bold')
axes[0, 0].set_ylabel('EUR/MWh')
axes[0, 0].xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

# Panel 2: Log returns
axes[0, 1].plot(returns.index, returns.values, color=COLORS['blue'], linewidth=0.4)
axes[0, 1].axhline(0, color=COLORS['gray'], linestyle='--', linewidth=0.5)
axes[0, 1].set_title('Log Returns (%)', fontweight='bold')
axes[0, 1].set_ylabel('Return (%)')
axes[0, 1].xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

# Panel 3: Distribution with Normal overlay
axes[1, 0].hist(returns.dropna(), bins=60, density=True, color=COLORS['blue'],
                alpha=0.7, edgecolor='none')
x = np.linspace(returns.min(), returns.max(), 200)
axes[1, 0].plot(x, norm.pdf(x, returns.mean(), returns.std()),
                color=COLORS['red'], linewidth=2, label='Normal')
axes[1, 0].set_title('Return Distribution (Fat Tails)', fontweight='bold')
axes[1, 0].set_xlabel('Return (%)')
axes[1, 0].legend(loc='upper center', bbox_to_anchor=(0.5, -0.10), frameon=False)

# Panel 4: Monthly averages
monthly_avg = prices.resample('M').mean()
axes[1, 1].bar(monthly_avg.index, monthly_avg.values, width=20,
               color=COLORS['purple'], alpha=0.7)
axes[1, 1].set_title('Monthly Average Price', fontweight='bold')
axes[1, 1].set_ylabel('EUR/MWh')
axes[1, 1].xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

fig.suptitle('Romanian Electricity — Exploratory Analysis', fontsize=14, fontweight='bold', y=1.02)
fig.tight_layout()
save_fig(fig, 'electricity_analysis.pdf')
plt.show()